In [1]:
import tensorflow as tf
import os
import PIL
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
"""os.environ["CUDA_VISIBLE_DEVICES"]="2"
logical_devices = tf.config.list_logical_devices('GPU') 
print(logical_devices)"""

'os.environ["CUDA_VISIBLE_DEVICES"]="2"\nlogical_devices = tf.config.list_logical_devices(\'GPU\') \nprint(logical_devices)'

In [2]:
DATA_SIZE = int(14e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

IMAGE_SIZE = (2792,99)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (99,2792,1) # shape qui rentre dans le cnn
# même taille pour toutes les parties d'images, présentes en même densité

INPUT_IMAGE_SHAPE_RGB = (99,2792,3) 
BATCH_SIZE = 4

THRESHOLD_PREDICTION = 0.5

NUM_CALLS = tf.data.experimental.AUTOTUNE

DATA_PATH = "/home/cogrannr/roues/MEFRO/grises/"
DATA_PATH_DEFAUTS_REELS = "/home/cogrannr/roues/MEFRO/images_defauts/"
DATA_PATH_ROUES_ENTIERES="/home/cogrannr/roues/MEFRO/Images_Roues/Actuelle/gris_blanc/"

# Preprocessing

In [3]:
"""
    Fonction custom, car doit être fait en eager execution, sinon c'est un param "none" passé à Image.open
    ouvereture des photos / resize
"""
def open_image(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)

    image_arr = image_arr[:,:,np.newaxis]
    return image_arr
    
def open_image_entiere(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)
    return image_arr

    
def parse_images(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def parse_images_entiere(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image_entiere,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label


def gray_to_rgb(image,label):
    image = image[:,:,0]
    image = tf.stack([image,image,image],axis=2)
    return image,label

def data_augmentation(image,label):
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=10.0/255.0)
    #image = tf.clip_by_value(image, 0.0, 1.0)

    return image,label
    
def reset_shapes_gray(image,label):
    image.set_shape(list(INPUT_IMAGE_SHAPE))
    return image, label


def reset_shapes_rgb(image,label):
    image.set_shape(list(INPUT_IMAGE_SHAPE_RGB))
    return image, label


In [4]:
# confusion matrix : 

def confusion_matrix(test_dataset,model, num_positive_class, num_negative_class):
    
    prediction = np.array([])
    labels = tf.constant([], dtype=tf.int32)

    for (data_batch, label_batch) in test_dataset:
        prediction = np.concatenate((prediction, model.predict_on_batch(data_batch)[:,0]))
        labels = tf.concat((labels, label_batch), axis=0)

    confusion_matrix = tf.math.confusion_matrix(
        labels=labels,predictions=tf.convert_to_tensor(prediction),num_classes=2)

    nomralized_matrix = tf.get_static_value(confusion_matrix)
    nomralized_matrix = nomralized_matrix.astype(np.float32)

    nomralized_matrix[0,:] = nomralized_matrix[0,:]/num_positive_class
    nomralized_matrix[1,:] = nomralized_matrix[1,:]/num_negative_class

    print("\nConfusion matrix : ")
    print(tf.get_static_value(confusion_matrix))
    print("\nNormalized confusion matrix : ")
    print(nomralized_matrix)

# Generate datasets

In [5]:
"""
    chargement des listes de fichiers photo
"""
def load_data(cover_dir,faulty_dir): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir+"/*",shuffle=False)
    file_names_cover = file_names_cover.take(DATA_SIZE//2)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir+"/*",shuffle=False)
    file_names_faulty = file_names_faulty.take(DATA_SIZE//2)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
   
    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
    
    return data_set

def load_data_entieres(cover_dir,faulty_dir, num_negative, num_positive): 

    file_names_cover = tf.data.Dataset.list_files(cover_dir,shuffle=False)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    data_set_cover = data_set_cover.take(num_positive)
    
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir,shuffle=False)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
    data_set_faulty = data_set_faulty.take(num_negative)

    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
    
    return data_set

""" 
    répartition des fichiers enn ensembles train/valid
"""
def generate_train_valid_test(cover_dir,faulty_dir, augment_data=False, gray_to_rgb_images=False,
                              entieres=False, num_positive=0,num_negative=0): 

    if entieres == True:
        data_set = load_data_entieres(cover_dir,faulty_dir, num_positive, num_negative)
    else:
        data_set = load_data(cover_dir, faulty_dir)

    train_dataset = data_set.take(DATA_TRAIN_SIZE)
    valid_dataset = data_set.skip(DATA_TRAIN_SIZE).take(DATA_VALID_SIZE)
    test_dataset = data_set.skip(DATA_TRAIN_SIZE+DATA_VALID_SIZE).take(DATA_TEST_SIZE)

    datasets_list = []
    
    for data_set in [train_dataset, valid_dataset, test_dataset]:

        if data_set == train_dataset:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
                
            if augment_data == True:
                data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
                data_set = data_set.map(data_augmentation, num_parallel_calls=NUM_CALLS)
        else:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
                            
        if gray_to_rgb_images == True:
            data_set =  data_set.map(gray_to_rgb, num_parallel_calls=NUM_CALLS)
            data_set = data_set.map(reset_shapes_rgb, num_parallel_calls=NUM_CALLS)
        else:
            data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.batch(BATCH_SIZE)
        data_set = data_set.prefetch(1)
        
        datasets_list.append(data_set)
        
    return datasets_list

# Model architecture

In [6]:
def init_model(base_model, architecture=1):

    # Inception v3
    if base_model == "inception":
        if architecture == 1: # train from scratch
            pretrained_model = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,
                                                                 input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = True
            model_input = pretrained_model.input

            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
            # output : 85x2048 : pooling + cv1 je pense + un FC pk pas
        
        if architecture == 2: # train last layers
            pretrained_model = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,
                                                                 input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = False
             
            for i in range(290,len(pretrained_model.layers)):
                pretrained_model.layers[i].trainable = True
            
            model_input = pretrained_model.input

            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
        
        if architecture == 3: # only last
            pretrained_model = tf.keras.applications.InceptionV3(weights="imagenet",include_top=False,
                                                                 input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = False
            
            model_input = pretrained_model.input
            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
    
    if base_model == "resnet":
        # Resnet 50
        if architecture == 1: # from scratch
            pretrained_model = tf.keras.applications.ResNet50V2(weights="imagenet",include_top=False,
                                                                input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = True
            model_input = pretrained_model.input

            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
            
        if architecture == 2: # last layers
            pretrained_model = tf.keras.applications.ResNet50V2(weights="imagenet",include_top=False,
                                                                input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = False
            
            num_of_layers = len(pretrained_model.layers)
            for i in range(num_of_layers-10, num_of_layers):
                pretrained_model.layers[i].trainable = True
            
            model_input = pretrained_model.input
            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
            
        if architecture == 3: # train last layer
            pretrained_model = tf.keras.applications.ResNet50V2(weights="imagenet",include_top=False,
                                                                input_shape=INPUT_IMAGE_SHAPE_RGB)
            pretrained_model.trainable = False
            model_input = pretrained_model.input

            x = pretrained_model.output
            flat = tf.keras.layers.Flatten()(x)            
            drop = tf.keras.layers.Dropout(0.5)(flat)
            out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)

    model = tf.keras.Model(inputs=model_input, outputs=out)
    print(model.summary())
    
    return model

# Training settings 

In [7]:
def train_model(model,train_dataset,valid_dataset, check_point_name, num_epochs, tensor_board_name=""):
    
    precision_metric = tf.keras.metrics.Precision(thresholds=THRESHOLD_PREDICTION)
    
    call_backs_list = [
        #tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience = 2),
        tf.keras.callbacks.ModelCheckpoint(
            filepath =check_point_name, monitor ="val_accuracy", save_best_only = True),
    ]
    
    if tensor_board_name != "":
        board = tf.keras.callbacks.TensorBoard(log_dir='./tensor_board_'+tensor_board_name)
        call_backs_list.append(board)
   
    optim = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  
    model.compile(loss='binary_crossentropy',optimizer=optim,
        metrics=[precision_metric,"accuracy"]) 
    
    model.fit(train_dataset,epochs=num_epochs,validation_data=valid_dataset,
              callbacks=call_backs_list)
    
    return model

#  Entrainement : Inception v3 : Parties de roues,  images gray


In [13]:
LEARNING_RATE = 0.0001 
NUM_EPOCHS = 5

DATA_SIZE = int(14e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)


NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

MODEL_NAME = "inception_gray_"
SAVE_PATH = "./trained_models/wheel_part/gray/"


In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope(): # freezed : derniere couche | scratch : tout train

    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts",
            augment_data=False, gray_to_rgb_images=True)

        model = init_model(base_model="inception",architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                                      check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", NUM_EPOCHS)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localho

Train for 2920 steps, validate for 973 steps
Epoch 1/5
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
   1/2920 [..............................] - ETA: 10:33:21WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.


UnknownError: 3 root error(s) found.
  (0) Unknown:  IsADirectoryError: [Errno 21] Is a directory: '/home/cogrannr/roues/MEFRO/grises/img_galbe'
Traceback (most recent call last):

  File "/home/etu/derouetl/venv_derouetl/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 234, in __call__
    return func(device, token, args)

  File "/home/etu/derouetl/venv_derouetl/lib/python3.6/site-packages/tensorflow_core/python/ops/script_ops.py", line 123, in __call__
    ret = self._func(*args)

  File "<ipython-input-5-f6d1d593cc83>", line 9, in open_image
    image = PIL.Image.open(name)

  File "/home/etu/derouetl/venv_derouetl/lib/python3.6/site-packages/PIL/Image.py", line 2809, in open
    fp = builtins.open(filename, "rb")

IsADirectoryError: [Errno 21] Is a directory: '/home/cogrannr/roues/MEFRO/grises/img_galbe'


	 [[{{node EagerPyFunc}}]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional_1]]
	 [[metrics/accuracy/div_no_na

In [16]:
for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts", gray_to_rgb_images=True)
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    
    print("evaluating inception model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

evaluating inception model for wheel part : galbe


AttributeError: 'NoneType' object has no attribute 'update'

##    Inception défauts réels, images gray

In [ ]:
SAVE_PATH = "./trained_models/wheel_part/gray/"

MODEL_NAME = "inception_gray_"

DATA_TEST_SIZE = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

for wheel_part in ["galbe", "ajours", "crochet"]:   #, "central"]:

    DATA_SIZE = DATA_TEST_SIZE[wheel_part]

    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2

    test_dataset = load_data(DATA_PATH+"img_"+wheel_part+"/*, 
    DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_avec_defauts/*")

    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)        

    test_dataset =  test_dataset.map(gray_to_rgb, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_rgb, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)

    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

# Train Inception Residus

In [ ]:
DATA_SIZE = int(146e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

LEARNING_RATE = 0.0001 
NUM_EPOCHS = 5

SAVE_PATH = "./trained_models/wheel_part/residus/"

MODEL_NAME = "inception_residus_"

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():

    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts",
            augment_data=False, gray_to_rgb_images=True)

        model = init_model(base_model="inception",architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                            check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5",NUM_EPOCHS)


In [ ]:
#test set residus
for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts",
            gray_to_rgb_images=True)
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating inception model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

## Test inception defauts réels  résidus


In [ ]:
DATA_SIZE_TEST = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

SAVE_PATH = "./trained_models/wheel_part/residus/"

MODEL_NAME = "inception_residus_"

In [ ]:

for wheel_part in ["galbe", "ajours", "crochet"]:   #, "central"]:

    DATA_SIZE = DATA_SIZE_TEST[wheel_part]

    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2
 
    test_dataset = load_data(DATA_PATH+"img_"+wheel_part+"_residus", 
            DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_residus_defauts")

    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)        

    test_dataset =  test_dataset.map(gray_to_rgb, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_rgb, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)

    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

# Entrainement : Resnet 50 , parties de roues, images gray

In [22]:
LEARNING_RATE = 0.0001 
NUM_EPOCHS = 5

DATA_SIZE = int(1e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

SAVE_PATH = "./trained_models/wheel_part/gray/"

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

MODEL_NAME = "resnet_gray_"

In [ ]:
#train from scratch

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():

    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts", gray_to_rgb_images=True)

        model = init_model(base_model="resnet",architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                                      check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", NUM_EPOCHS)

In [23]:
for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts", gray_to_rgb_images=True)
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating resnet model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

evaluating resnet model for wheel part : galbe
50/50 [==============================] - 34s 684ms/step - loss: 0.0242 - precision_1: 0.9892 - accuracy: 0.9950
evaluating resnet model for wheel part : ajours
50/50 [==============================] - 31s 627ms/step - loss: 0.0043 - precision_2: 1.0000 - accuracy: 1.0000
evaluating resnet model for wheel part : crochet
50/50 [==============================] - 36s 718ms/step - loss: 0.0426 - precision_3: 1.0000 - accuracy: 0.9900
evaluating resnet model for wheel part : central
50/50 [==============================] - 34s 686ms/step - loss: 0.1471 - precision_4: 0.9882 - accuracy: 0.9550


## Resnet défauts réels  images gray

In [ ]:
DATA_SIZE_TEST = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

SAVE_PATH = "./trained_models/wheel_part/gray/"

MODEL_NAME = "resnet_gray_"


In [ ]:
for wheel_part in ["galbe", "ajours", "crochet"]:   #, "central"]:

    DATA_SIZE = DATA_SIZE_TEST[wheel_part]
    
    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2

    test_dataset = load_data(DATA_PATH+"img_"+wheel_part+"/*, 
    DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_avec_defauts/*")

    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)        

    test_dataset =  test_dataset.map(gray_to_rgb, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_rgb, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)

    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

# Train Resnet résidus

In [13]:
DATA_SIZE = int(5e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

SAVE_PATH = "./trained_models/wheel_part/residus/"

MODEL_NAME = "resnet_residus_"

LEARNING_RATE = 0.0001 
NUM_EPOCHS = 7

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", 
            DATA_PATH+"img_"+wheel_part+"_residus_defauts",
            augment_data=False, gray_to_rgb_images=True)

        model = init_model(base_model="resnet",architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                        check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", NUM_EPOCHS)


In [9]:
# test sur defauts fabriqués
for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts", 
        gray_to_rgb_images=True)
    
    check_point= tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating resnet model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

evaluating resnet model for wheel part : galbe
50/50 [==============================] - 36s 720ms/step - loss: 0.0070 - precision_1: 1.0000 - accuracy: 1.0000
evaluating resnet model for wheel part : ajours
50/50 [==============================] - 34s 672ms/step - loss: 0.0742 - precision_2: 1.0000 - accuracy: 0.9800
evaluating resnet model for wheel part : crochet
50/50 [==============================] - 36s 728ms/step - loss: 0.0606 - precision_3: 0.9889 - accuracy: 0.9800
evaluating resnet model for wheel part : central
50/50 [==============================] - 33s 660ms/step - loss: 0.2571 - precision_4: 0.9740 - accuracy: 0.9050


In [14]:
print(SAVE_PATH,MODEL_NAME)

for name in ["galbe", "ajours", "crochet", "central"]:
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+name+".h5")
    print("model ",name," loaded")
    print(check_point)
    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts", 
            gray_to_rgb_images=True)
        print(test_dataset)
        
        print("evaluate model",name," on data : ",wheel_part)
        check_point.evaluate(test_dataset, verbose=1)


./trained_models/wheel_part/residus/ resnet_residus_
model  galbe  loaded
<PrefetchDataset shapes: ((None, 99, 2792, 3), (None,)), types: (tf.float32, tf.int32)>
evaluate model galbe  on data :  galbe
250/250 [==============================] - 171s 685ms/step - loss: 0.0349 - precision_1: 0.9922 - accuracy: 0.9920
<PrefetchDataset shapes: ((None, 99, 2792, 3), (None,)), types: (tf.float32, tf.int32)>
evaluate model galbe  on data :  ajours
250/250 [==============================] - 165s 658ms/step - loss: 350.8412 - precision_1: 0.5120 - accuracy: 0.5120
<PrefetchDataset shapes: ((None, 99, 2792, 3), (None,)), types: (tf.float32, tf.int32)>
evaluate model galbe  on data :  crochet
250/250 [==============================] - 170s 681ms/step - loss: 58.4650 - precision_1: 0.5120 - accuracy: 0.5120
<PrefetchDataset shapes: ((None, 99, 2792, 3), (None,)), types: (tf.float32, tf.int32)>
evaluate model galbe  on data :  central
250/250 [==============================] - 162s 648ms/step - loss

## Test defauts reels residus

In [ ]:
DATA_SIZE_TEST = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

SAVE_PATH = "./trained_models/wheel_part/residus/"

MODEL_NAME = "resnet_residus_"

In [ ]:
for wheel_part in ["galbe", "ajours", "crochet"]:# "central"]:

    DATA_SIZE = DATA_SIZE_TEST[wheel_part]
    
    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2

    test_dataset = load_data(DATA_PATH+"img_"+wheel_part+"_residus/*", 
            DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_residus_defauts/*")

    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)        

    test_dataset =  test_dataset.map(gray_to_rgb, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_rgb, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)

    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)
    

# Train roues entieres

#    Inception




In [ ]:
IMAGE_SIZE = (1624,1234)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (1234,1624,3) # shape qui rentre dans le cnn
    #roues deja en rgb

DATA_SIZE = int(5524) # taille des données (test/valid/test compris), max : 74533;    5524 : rapport 1:4
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

NUM_POSITIVE_CLASS_TOTAL = 1381 # constant
NUM_NEGATIVE_CLASS_TOTAL = DATA_SIZE - NUM_POSITIVE_CLASS_TOTAL

NUM_POSITIVE_CLASS = int(NUM_POSITIVE_CLASS_TOTAL*0.2) #276 constant
NUM_NEGATIVE_CLASS = int(NUM_NEGATIVE_CLASS_TOTAL*0.2) # 828 pour 1:4    ; 14630 pour tout 

SAVE_PATH = "./trained_models/whole_wheel/"
MODEL_NAME = "inception_whole_wheel_1.h5"   # train 1:4  (1)   et 1:7000 (2)

LEARNING_RATE = 0.0001
NUM_EPOCHS = 5
BATCH_SIZE = 6


In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():

    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
        faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
        augment_data=True, gray_to_rgb_images=False,entieres=True,
        num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


    model = init_model(base_model="inception",architecture=1)
    model = train_model(model, train_dataset, valid_dataset,
                                  check_point_name=SAVE_PATH+MODEL_NAME,NUM_EPOCHS)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
cover 73184
faulty 1369
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then bro

Train for 4970 steps, validate for 1656 steps
Epoch 1/5
INFO:tensorflow:batch_all_reduce: 190 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 190 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
3422/4970 [===================>..........] - ETA: 22:27 - loss: 0.1159 - precision: 0.7343 - accuracy: 0.9896

In [7]:
# test sur defauts fabriqués

train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
    augment_data=False, gray_to_rgb_images=True,entieres=True,
    num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME)
print("evaluating Inception model: ")
check_point.evaluate(test_dataset, verbose=1)

confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)


evaluating Inception model for inception: 
1656/1656 [==============================] - 1062s 641ms/step - loss: 56.6259 - precision: 0.0000e+00 - accuracy: 0.9802


[56.62591324221109, 0.0, 0.98020667]

# Resnet


In [ ]:
IMAGE_SIZE = (1624,1234)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (1234,1624,3) # shape qui rentre dans le cnn
    #roues deja en rgb

DATA_SIZE = int(5524) # taille des données (test/valid/test compris), max : 74533;    5524 : rapport 1:4
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

NUM_POSITIVE_CLASS_TOTAL = 1381 # constant
NUM_NEGATIVE_CLASS_TOTAL = DATA_SIZE - NUM_POSITIVE_CLASS_TOTAL

NUM_POSITIVE_CLASS = int(NUM_POSITIVE_CLASS_TOTAL*0.2) #276 constant
NUM_NEGATIVE_CLASS = int(NUM_NEGATIVE_CLASS_TOTAL*0.2) # 828 pour 1:4    ; 14630 pour tout 

SAVE_PATH = "./trained_models/whole_wheel/"
MODEL_NAME = "resnet_whole_wheel_1.h5"   # train 1:4  (1)   et 1:7000 (2)

LEARNING_RATE = 0.0001
NUM_EPOCHS = 5
BATCH_SIZE = 6


In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():

    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
        faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
        augment_data=True, gray_to_rgb_images=False,entieres=True,
        num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


    model = init_model(base_model="resnet",architecture=1)
    model = train_model(model, train_dataset, valid_dataset,
                                  check_point_name=SAVE_PATH+MODEL_NAME,NUM_EPOCHS)



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


In [8]:
# test sur defauts fabriqués

train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
    cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
    faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
    augment_data=True, gray_to_rgb_images=True,entieres=True,
    num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME)
print("evaluating Inception model for resnet: ")
check_point.evaluate(test_dataset, verbose=1)

confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)



evaluating Inception model for resnet: 
1656/1656 [==============================] - 1049s 634ms/step - loss: 103.4730 - precision: 0.0000e+00 - accuracy: 0.9803


[103.47300268716859, 0.0, 0.98034084]